## Importing and loading datasets

In [103]:
# importing libraries 
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns



In [104]:
# load csv files
budgets = pd.read_csv("/Users/Maureen /Documents/Phase2-project/datasets/tn.movie_budgets.csv")
tmdb = pd.read_csv("/Users/Maureen /Documents/Phase2-project/datasets/tmdb.movies.csv")


# Connect to SQLite database
conn = sqlite3.connect("/Users/Maureen /Documents/Phase2-project/datasets/im.db")




## Data Overview

In [105]:
# Explore CSV

# Preview the first few rows
print(budgets.head())
print(tmdb.head())

# Check column names
print(budgets.columns)
print(tmdb.columns)

# Check basic info
print(budgets.info())
print(tmdb.info())

# Describe numeric columns
print(budgets.describe())
print(tmdb.describe())

# missing value 
print(budgets.isnull().sum())
print(tmdb.isnull().sum())


   id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   
3   4   May 1, 2015                      Avengers: Age of Ultron   
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   

  production_budget domestic_gross worldwide_gross  
0      $425,000,000   $760,507,625  $2,776,345,279  
1      $410,600,000   $241,063,875  $1,045,663,875  
2      $350,000,000    $42,762,350    $149,762,350  
3      $330,600,000   $459,005,868  $1,403,013,963  
4      $317,000,000   $620,181,382  $1,316,721,747  
   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en   
3           3      [1

In [106]:
# Explore SQLite database

# All tables in the database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in database:", tables)

# Load IMDb tables
movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)

conn.close()

Tables in database:             name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers
8   tmdb_budgets


In [107]:
print(budgets.columns)

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')


In [108]:
print(tmdb.columns)

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')


## Data cleaning 
Movie budgets dataset

In [109]:
# Remove $ and commas, then convert to integers
for col in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    budgets[col] = budgets[col].replace('[\$,]', '', regex=True).astype(int)

#Convert release_date to datetime
budgets['release_date'] = pd.to_datetime(budgets['release_date'])

# Check for duplicates
print("Number of duplicate rows:", budgets.duplicated().sum())

# Check for movies with 0 or negative budgets
invalid_budget = budgets[budgets['production_budget'] <= 0]
print("Invalid production budgets:\n", invalid_budget)

# Filter out invalid rows (budget <= 0 or negative gross)
budgets = budgets[(budgets['production_budget'] > 0) & 
                  (budgets['domestic_gross'] >= 0) & 
                  (budgets['worldwide_gross'] >= 0)]

Number of duplicate rows: 0
Invalid production budgets:
 Empty DataFrame
Columns: [id, release_date, movie, production_budget, domestic_gross, worldwide_gross]
Index: []


 TMDB dataset

In [110]:


# Drop duplicates
tmdb.drop_duplicates(inplace=True)

# Convert release_date to datetime
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'], errors='coerce')

# Drop rows with missing release_date or title
tmdb.dropna(subset=['title', 'release_date'], inplace=True)

# Extract release year for merging
tmdb['release_year'] = tmdb['release_date'].dt.year

# Drop unnecessary column
tmdb.drop(columns=['Unnamed: 0'], inplace=True)

# Keep only useful columns
tmdb = tmdb[['id', 'title', 'original_language', 'popularity', 
             'vote_average', 'vote_count', 'release_year', 'genre_ids']]


tmdb.head()


,id,title,original_language,popularity,vote_average,vote_count,release_year,genre_ids
0,12444,Harry Potter and the Deathly Hallows: Part 1,en,33.533,7.7,10788,2010,"[12, 14, 10751]"
1,10191,How to Train Your Dragon,en,28.734,7.7,7610,2010,"[14, 12, 16, 10751]"
2,10138,Iron Man 2,en,28.515,6.8,12368,2010,"[12, 28, 878]"
3,862,Toy Story,en,28.005,7.9,10174,1995,"[16, 35, 10751]"
4,27205,Inception,en,27.920,8.3,22186,2010,"[28, 878, 12]"


Merging

In [111]:
# Create release year from release_date
budgets['release_year'] = budgets['release_date'].dt.year

# Clean movie titles for merging
budgets['movie'] = budgets['movie'].str.strip().str.lower()
tmdb['title'] = tmdb['title'].str.strip().str.lower()


In [112]:
# Merge datasets
merged = pd.merge(
    budgets,
    tmdb,
    left_on=['movie', 'release_year'],  # merge on movie title and release year
    right_on=['title', 'release_year'],
    how='inner'           # keep movies present in both datasets
)
# Preview 
print("Merged dataset shape:", merged.shape)
merged.head()



Merged dataset shape: (1811, 14)


,id_x,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,id_y,title,original_language,popularity,vote_average,vote_count,genre_ids
0,1,2009-12-18,avatar,425000000,760507625,2776345279,2009,19995,avatar,en,26.526,7.4,18676,"[28, 12, 14, 878]"
1,2,2011-05-20,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,2011,1865,pirates of the caribbean: on stranger tides,en,30.579,6.4,8571,"[12, 28, 14]"
2,4,2015-05-01,avengers: age of ultron,330600000,459005868,1403013963,2015,99861,avengers: age of ultron,en,44.383,7.3,13457,"[28, 12, 878]"
3,7,2018-04-27,avengers: infinity war,300000000,678815482,2048134200,2018,299536,avengers: infinity war,en,80.773,8.3,13948,"[12, 28, 14]"
4,9,2017-11-17,justice league,300000000,229024295,655945209,2017,141052,justice league,en,34.953,6.2,7510,"[28, 12, 14, 878]"
